**Dénes Csala**  
University of Bristol, 2022  

Based on *Elements of Data Science* ([Allen B. Downey](https://allendowney.com), 2021) and *Python Data Science Handbook* ([Jake VanderPlas](https://jakevdp.github.io/PythonDataScienceHandbook/), 2018)

License: [MIT](https://mit-license.org/)

# ONS data load

Load an ONS data API url

In [ ]:
import requests, pandas as pd

In [ ]:
url ='https://api.beta.ons.gov.uk/v1/datasets/index-private-housing-rental-prices/editions/time-series/versions/13/observations?time=*&geography=E12000007&indexandyearchange=index'
html = requests.get(url)
json_data = html.json()

The response returned `json_data` is a dictionary. The values that concern us are stored under the `['observations']` key. Let's save this out into the `observations` variable.

In [ ]:
observations=json_data['observations']

Now we iterate through all the elements `x` of the `observations` list. We will also initilise an empty list `empty_data`. Then, at every step, we will save only the sub-keys of the `x` that are of interest to us - in this case `x['dimensions']['Time']['label']` and `x['observation']`. We save these values under keys `time` and `value` into a helpder dictionary called `dummy`. We then append this dictionary to the `empty_list` that we have created. Likewise, with every step, the `empty_list` gains a new element, equalt to the contents of the `dummy` dictionary at that step.

In [ ]:
empty_data=[]
for x in observations:
  # print(x['dimensions']['Time']['label'],x['observation'])
  dummy={'time': x['dimensions']['Time']['label'],
         'value': x['observation']
         }
  empty_data.append(dummy)

Great. Now we have a list of dictionaries - the _standard form_ of data! WE caneasily convert this to a _pandas_ `DataFrame`.

In [ ]:
df = pd.DataFrame(empty_data)

In [ ]:
df.head(3)

,time,value
0,Feb-11,86.6
1,Apr-11,87.1
2,Sep-13,96.8


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   time    201 non-null    object
 1   value   201 non-null    object
dtypes: object(2)
memory usage: 3.3+ KB


We can see that the type of the columns is `object` - which means `string`. Let's do some conversions.

In [ ]:
df['value']=df['value'].astype(float)

In [ ]:
df.head(3).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    3 non-null      object 
 1   value   3 non-null      float64
dtypes: float64(1), object(1)
memory usage: 176.0+ bytes


In [ ]:
import numpy as np

In [ ]:
df['month']=df['time'].str.split('-').str[0]
df['year']=df['time'].str.split('-').str[1]
df['year']='20'+df['year']
df['day']='01'

In [ ]:
df.head(2)

,time,value,month,year,day
0,Feb-11,86.6,Feb,2011,01
1,Apr-11,87.1,Apr,2011,01


In [ ]:
df['date']=pd.to_datetime(df['year'] + '-' +df['month'] + '-' +df['day'])

Now the `date` column is of the `datetime` type. This means we can use datetime functions on it, such as:

In [ ]:
df['date'].dt.month_name()

0       February
1          April
2      September
3      September
4        January
         ...    
196         July
197      October
198      January
199        March
200      January
Name: date, Length: 201, dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    201 non-null    object        
 1   value   201 non-null    float64       
 2   month   201 non-null    object        
 3   year    201 non-null    object        
 4   day     201 non-null    object        
 5   date    201 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 9.5+ KB


We are happy with this format. However, you can see that the `date` column has an `[ns]` at the end of the type. This is codename for _not serializable`. This means it does not follow the `JSON` standard. This is because we used the _numpy_ package to create these dates in _python_. In _Javascript_, a dirrenet date format is used. Therefore, in order to export to _Vega_, we need to convert this back to string.

In [ ]:
df['date']=df['date'].astype(str)

Ready to export. In _CSV_:

In [ ]:
df.to_csv('my_data.csv')

To _JSON_.

In [ ]:
import json

In [ ]:
json_list_of_dicts=list(df.T.to_dict().values())
open('my_data.json','w').write(json.dumps(json_list_of_dicts))

20583